In [1]:
#1
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#2
df = pd.read_csv("cyberbullying.csv")
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [3]:
#3

# Assuming your DataFrame is named df
# Replace 'your_dataframe' with your actual DataFrame name

df['words'] = df['tweet_text'].apply(lambda x: x.split())

In [4]:
df.head()

,tweet_text,cyberbullying_type,words
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"[In, other, words, #katandandre,, your, food, ..."
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,"[Why, is, #aussietv, so, white?, #MKR, #theblo..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,"[@XochitlSuckkks, a, classy, whore?, Or, more,..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,"[@Jason_Gio, meh., :P, thanks, for, the, heads..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,"[@RudhoeEnglish, This, is, an, ISIS, account, ..."


In [10]:
uniquewords=df['words'].explode().unique()

In [11]:
len(uniquewords)

118445

In [14]:
#
for taco in range(len(df['words'])):
    for burrito in range(len(df['words'][taco])):
        x=1

In [17]:
unique_words_df = pd.DataFrame({'word': df['words'].explode().unique()})
unique_words_df['word_number'] = range(1, len(unique_words_df) + 1)

# Merge the original DataFrame with the unique word DataFrame
df = df.merge(unique_words_df, left_on='words', right_on='word', how='left')

# Drop unnecessary columns and reset index
df = df.drop(['word', 'words'], axis=1).sort_values(by=['tweet_text']).reset_index(drop=True)

TypeError: unhashable type: 'list'